# Batched, Multi-Input, Multi-Output Gaussian Process Regression with GPyTorch
In this notebook, we will look at the use of batched, multi-dimensional GPyTorch (`gpytorch`) models for Gaussian Process Regression tasks.

Our aim is to focus on code optimization, as well as discussing the use of certain tools such as `gc` that can be used to ensure optimal hardware performance and reassure against memory leaks.

## Import Block

In [ ]:
# GPyTorch, NumPy, Scikit-Learn, Matplotlib
import gpytorch
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
from mpl_toolkits.mplot3d import Axes3D

# Timing and math
import time
import math
import os
import psutil
import gc
process = psutil.Process(os.getpid())

## Define Batched GPyTorch Model
To model different outcomes over multi-dimensional targets, we will encode this by batching the model.  Many `gpytorch` models have batching capability in the form of specifying the keyword argument `batch_shape` (which expects a `torch.Size([x])` object).

In [ ]:
## Define the Model
class BatchedGP(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood, shape):
        super(BatchedGP, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean(batch_shape=torch.Size([shape]))
        self.covar_module = gpytorch.kernels.ScaleKernel(
            gpytorch.kernels.MaternKernel(batch_shape=torch.Size([shape])),
            batch_shape=torch.Size([shape])
        )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

## Training Function
The function below defines the function used for training our multidimensional GPyTorch model.  To batch over both the multidimensional targets/outputs as well as along the standard batch axes, we extend the first dimension by tiling the inputs by the number of output dimensions, and resizing the targets such that each block of the training and predicted targets is a dimension of the targets over a larger batch of points.

Note the use of the following techniques to ensure/mitigate the possibility of GPU memory leak:

1. Defining a separate function, with its own scope, for training, and calling this function.  This is similar to a `closure()` function, which you may see for other `torch` optimizers such as `LBFGS`. 

2. Using the garbage collection (`gc`) package, and calling `gc.collect()` at the end of the training function.

3. Ensuring that anything we pull from the computational graph, e.g. `loss.item()`, is removed from the computational graph by calling `.item()` to avoid the graph persisting after it goes out of context.

In [ ]:
def train_gp_batched_scalar(Zs, Ys, use_cuda=False, epochs=10, lr=0.1):
    """Computes a Gaussian Process object using GPyTorch. Each outcome is
    modeled as a single scalar outcome.
    """
    # Preprocess batch data
    B, N, XD = Zs.shape  # B is batch dimension, N is points dimension, and XD is feature dimension
    YD = Ys.shape[-1]  # YD is target dimension
    
    # Convert features to tensor, and tile according to output targets
    train_x = torch.tensor(Zs)
    train_x = train_x.repeat((YD, 1, 1))
    
    # Convert targets to tensor, and reshape from (B, N, YD) --> (B * YD, N)
    train_y = torch.tensor(Ys)
    train_y = train_y.view(B * YD, N)

    # initialize likelihood and model - batch over output dimensions and batches
    likelihood = gpytorch.likelihoods.GaussianLikelihood(batch_shape=torch.Size([B * YD]))
    model = BatchedGP(train_x, train_y, likelihood, B * YD)

    # Find optimal model hyperparameters
    model.train()
    likelihood.train()

    # Use the adam optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    # "Loss" for GPs - the marginal log likelihood
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)
    
    # If using GPU
    if use_cuda:
        model = model.cuda()
        likelihood = likelihood.cuda()
        train_x = train_x.cuda()
        train_y = train_y.cuda()
        mll = mll.cuda()
    
    # Define training helper function
    def epoch_train():
        optimizer.zero_grad()  # Zero gradients
        output = model(train_x)  # Compute noise-free output
        loss = -mll(output, train_y).sum()  # Compute batched loss
        loss.backward()  # Compute gradients with backpropagation
        optimizer.step()  # Update weights with gradients
        optimizer.zero_grad()  # Zero gradients
        gc.collect()  # Used to ensure there is no memory leak
    
    # Run training
    for i in range(epochs):
        epoch_train()

    torch.cuda.empty_cache()
    return model, likelihood

## Run Training
With our model, likelihood, and training routine defined, we are now ready to train!

In [ ]:
# Set parameters
B = 256  # Batch dimension
N = 25   # Number of points
XD = 2   # Dimension of features
YD = 1   # Dimension of targets
EPOCHS = 50
USE_CUDA = torch.cuda.is_available()

# Create training data and labels
train_x_np = np.random.normal(loc=0, scale=1, size=(B, N, XD))  # Create as np array
train_y_np = np.random.normal(loc=0, scale=1, size=(B, N, YD))  # Create as np array
train_x = torch.tensor(train_x_np).float()  # Can use .double() for 64-bit fp precision
train_y = torch.tensor(train_y_np).float()  # Can use .double() for 64-bit fp precision

# Time GPyTorch training
model, likelihood = train_gp_batched_scalar(train_x_np, train_y_np,
                        use_cuda=USE_CUDA, epochs=EPOCHS, lr=0.1)

# Calling .eval() places the model and likelihood in "posterior" mode
model.eval()
likelihood.eval()

# Define mll
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)